# Import

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from src.particle import *
from src.tracker import *
from src.widgets import *
import warnings
warnings.filterwarnings('ignore')

SyntaxError: invalid syntax (tracker.py, line 89)

# Tracking

In [ ]:
def track_values(p, charge=1., B=1.):
    return p/(charge*B)

In [ ]:
# load in data

data = pd.read_hdf('EEEE/0_allcomb_0.h5')
coords = [f'{i}' for i in np.arange(0, 6624)]
hits = data[coords]
momenta = data.loc[:,['phi', 'pt']]
momenta.loc[:,'charge'] = data.loc[:,'pdg']/abs(data.loc[:,'pdg'])
momenta.loc[:,'phi'] = momenta.loc[:,'phi']*np.pi/180.
momenta.reset_index(inplace = True)
momenta

In [ ]:
tw = TrackingWidget(momenta.loc[:1,:], B = 0.1)

In [ ]:
tw.show()

In [ ]:
#trying to calculate ecl hit, ignore this please
def find_crystal(radius, phi):
    x = radius*np.cos(phi)
    y = radius*np.sin(phi)
    d=math.sqrt(x**2 + y**2)
    a=(139**2-radius**2+d**2)/(2*d)
    h=math.sqrt(abs(radius**2-a**2))
    x2=a*(x)/d   
    y2=a*(y)/d   
    x3=x2+h*(y)/d     
    y3=y2-h*(x)/d 

    x4=x2-h*(y)/d
    y4=y2+h*(x)/d


    rho = math.sqrt(x4**2 + y4**2)
    theta = np.arctan2(y4,x4)

    if theta < 0:
        theta += 2*np.pi
        
    return theta

radius = 21.0
phi = 2.86

find_crystal(radius, phi)

In [ ]:
class ECLCrystal:
    def __init__(self, x, y, edge, hit=False):
        self.x = x
        self.y = y
        self.edge = edge
        self.hit = hit
        self.chosen = False
        self.facecolor = to_rgba("white")
        self.edgecolor = to_rgba("black")
        
    def draw(self,ax):
        ax.add_patch(Rectangle((self.x, self.y), self.edge, self.edge, edgecolor = self.edgecolor, facecolor = self.check_color(), linewidth = 1))
        
    def get_patch(self):
        return Rectangle((self.x, self.y), self.edge, self.edge, edgecolor = self.edgecolor, facecolor = self.check_color(), linewidth = 1)
    
    def check_color(self):
        if self.chosen:
            return "red"
        elif self.hit:
            return "red"
        else:
            return "grey"

In [ ]:
class ECal:
    def __init__(self, nrows, ncols, hits, crystal_edge = 1):
        self.crystals = np.empty((nrows, ncols), dtype = ECLCrystal)
        hit = False
        #he did this just to confuse the fuck out of me
        for c in range(ncols):
            for r in range(nrows):
                if hits[144*c+r] > 0.0:
                    hit = True
                self.crystals[r,c] = ECLCrystal(r*crystal_edge, c*crystal_edge, crystal_edge-0.2, hit=hit)
                hit = False
    
    def draw(self, ax):
        for r in self.crystals:
            for c in r:
                c.draw(ax)
    
    def draw_collection(self,ax):
        ax.add_collection(self.collection)
                
    def get_coords(self):
        coords = []
        for cols in self.crystals:
            for crystal in cols:
                coords.append([crystal.x, crystal.y])
        return coords
    
    def get_collection(self):
        crystals = []
        for c in self.crystals.flatten():
            crystals.append(c.get_patch())
        self.collection = PatchCollection(crystals, match_original=True)
        return self.collection
    
    def get_crystal(self, rectangle):
        x,y = rectangle.get_xy()
        for i,r in enumerate(self.crystals):
            for j,c in enumerate(r):
                if x == c.x and y==c.y:
                    return i,j

## V2

In [ ]:
import numpy as np

from matplotlib.widgets import LassoSelector
from matplotlib.path import Path


class SelectFromCollection:

    def __init__(self, ax, ecal):
        self.canvas = ax.figure.canvas
        self.ecal = ecal    
        
        xy = np.array(ecal.get_coords()).T      
        self.ecal.get_collection()
        self.ecal.draw_collection(ax)
        self.collection = ax.scatter(xy[0], xy[1], s=0)
        self.xys = self.collection.get_offsets()
        self.Npts = len(self.xys)
    
        # Ensure that we have separate colors for each object
        self.fc = self.ecal.collection.get_edgecolors()
        if len(self.fc) == 0:
            raise ValueError('Collection must have a facecolor')
        elif len(self.fc) == 1:
            self.fc = np.tile(self.fc, (self.Npts, 1))
        self.lasso = LassoSelector(ax, onselect=self.onselect)
        self.ind = []

    def onselect(self, verts):
        path = Path(verts)
        self.ind = np.nonzero(path.contains_points(self.xys))[0]
        self.fc[:] = to_rgba("Black")
        self.fc[self.ind] = to_rgba("green")
        self.ecal.collection.set_edgecolors(self.fc)
        self.canvas.draw_idle()

In [ ]:
ecal = ECal(144,46,hits.iloc[0].to_numpy())
subplot_kw = dict(xlim=(-10,150), ylim=(-10,50), autoscale_on=False)
fig, ax = plt.subplots(figsize=(15,10), subplot_kw=subplot_kw)

selector = SelectFromCollection(ax, ecal)
#selector.onselect([[0,10],[10,10],[9,10],[0,1]])

def accept(event):
    if event.key == "enter":
        print("Selected points:")
        print(selector.xys[selector.ind])
        selector.disconnect()
        ax.set_title("")
        fig.canvas.draw()

fig.canvas.mpl_connect("key_press_event", accept)
ax.set_title("Press enter to accept selected points.")

plt.show()

In [ ]:
ecal.collection.get_facecolors()